In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

# 🚀 99_Setup_Workflow Notebook

%md
### 🛠 99_Setup_Workflow.ipynb
This notebook creates or replaces a Databricks Workflow for the AeroDemo project.
It chains together the 01_ and 02_ series notebooks, using a fresh ML-enabled, autoscaling cluster (1–6 workers) for each task.

✅ Uses Databricks Runtime ML (latest: 14.3.x-cpu-ml-scala2.12)
✅ Configures autoscale (1–6 workers)
✅ Auto-terminates after 60 minutes idle
---

### 📋 Current notebooks included:
<strike>✅ `01_Table_Creation.ipynb`</strike>  
✅ `02_01_Sensor_Data_Generation.ipynb`  
✅ `02_02_Engine_Data_Generation.ipynb`  
✅ `02_03_CabinPressurization_Data_Generation.ipynb`  
✅ `02_04_Airframe_Synthetic_Data_Generation.ipynb`  
✅ `02_05_LandingGear_Data_Generation.ipynb`  
✅ `02_06_Avionics_Data_Generation.ipynb`  
✅ `02_07_ElectricalSystems_Data_Generation.ipynb`  
✅ `02_08_FuelSystems_Data_Generation.ipynb`  
✅ `02_09_HydraulicSystems_Data_Generation.ipynb`  
✅ `02_10_EnvironmentalSystems_Data_Generation.ipynb`

---

### 🔗 Future additions:
We’ll expand this workflow to include:
- `03_` series (DLT pipelines)
- `04_` series (ML models + scoring)
- `05_` series (dashboarding + alerts)

### ⚙️ Important Setup Notes

✅ **Cluster setup**
- Replace `<YOUR_CLUSTER_ID>` in the script with:
  - Your existing Databricks cluster ID, **or**
  - Switch to `new_cluster` configuration if you want the workflow to create its own cluster

✅ **Repo + notebook paths**
- Make sure all notebook paths align with:
  `/Repos/honnuanand/databricks-aerodemo/<NOTEBOOK_NAME>.ipynb`

✅ **Databricks SDK**
- This script uses the `databricks-sdk` (Python client).
- Run it from:
  - A Databricks notebook, **or**
  - A local Python environment with `databricks-sdk` installed and configured


---

Once you run this, you’ll have a fresh **AeroDemo_DataPipeline** workflow  
that orchestrates all current synthetic data generation steps!

### 🛠 Workflow Setup Helper Notes

This script sets up a Databricks Workflow that runs the AeroDemo synthetic data pipeline notebooks.

---

✅ **Configurable Parameters**
- `NOTEBOOK_BASE_PATH` → Set to the full workspace path where your notebooks live.  
  Example: `/Workspace/Users/anand.rao@databricks.com/databricks-aerodemo`

- `CLUSTER_ID` → Replace with:
  - An **existing cluster ID** you want the workflow to run on, **or**
  - Replace `existing_cluster_id` with a `new_cluster` configuration block if you want the workflow to create its own cluster

- `WORKFLOW_NAME` → Choose a descriptive name for your Databricks Job (Workflow).

---

✅ **Notebook Path Pattern**
- Each task points to:  
  `{NOTEBOOK_BASE_PATH}/{NOTEBOOK_NAME}.ipynb`

Make sure your notebook filenames in the workspace exactly match those listed in the `notebooks` array.

---

✅ **Expansion**
- You can later add:
  - `03_` series (DLT ingestion pipelines)
  - `04_` series (ML model training + scoring)
  - `05_` series (visualization + alert notebooks)

Just extend the `notebooks` list in the code and rerun the script to update the workflow.

---

✅ **Execution**
- Run this script inside a Databricks notebook or from a local Python environment with the `databricks-sdk` properly configured.

In [0]:
import importlib
from config import aerodemo_config

importlib.reload(aerodemo_config)  # Force reload of the module
from config.aerodemo_config import get_config  # Import the get_config function

# Create a text widget for environment selection
dbutils.widgets.dropdown("ENV", "dev", ["dev", "staging", "prod"], "Environment")
env = dbutils.widgets.get("ENV")
config = get_config(env)
print(f"Using environment: {env}")
print(f"Catalog: {config['catalog']}")

In [0]:
from config.aerodemo_config import get_config

# Retrieve configuration for the current environment
env = dbutils.widgets.get("ENV")
config = get_config(env)

print(f"Using environment: {env}")
print(f"Catalog: {config['catalog']}")
print(f"Schema: {config['schema']}")
print(f"Databricks Instance: {config['databricks_instance']}")
print(f"Personal Access Token: {config['pat_token']}")
print(f"E2E Workflow Job ID: {config['e2e_workflow_job_id']}")


In [0]:
import os

# Get current notebook path and folder
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
folder_path = os.path.dirname(notebook_path)
workspace_path = f"/Workspace{folder_path}"

print(f"✅ Notebook path: {notebook_path}")
print(f"✅ Notebook folder (user-level): {folder_path}")
print(f"✅ Notebook folder (workspace-level): {workspace_path}")

In [0]:
import os
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.jobs import Task, NotebookTask, PipelineTask, TaskDependency
from config.aerodemo_config import get_config

# Get current notebook context
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
folder_path = os.path.dirname(notebook_path)

# Dynamically resolve project base paths
workspace_base_path = f"/Workspace{folder_path}"
project_root = "/".join(workspace_base_path.split("/")[:-1])

synthetic_data_path = f"{project_root}/02_SyntheticData"
feature_registration_path = f"{project_root}/03B_FeatureRegistration"
workflows_path = f"{project_root}/03_WorkflowsAndPipelines"

# Retrieve configuration
env = dbutils.widgets.get("ENV")
config = get_config(env)

print(f"✅ Using project root: {project_root}")

# ---------- CONFIG ----------
WORKFLOW_NAME = config['workflow_configs']['workflow_name']
EXISTING_CLUSTER_ID = config['workflow_configs']['existing_cluster_id']
DLT_PIPELINE_ID = config['workflow_configs']['dlt_pipeline_id']

# List of synthetic notebooks (parallel)
synthetic_notebooks = [
    "02_01_Synthetic_Data_Generation_v2",
    "02_02_Engine_Data_Generation",
    "02_03_CabinPressurization_Data_Generation",
    "02_04_Airframe_Synthetic_Data_Generation",
    "02_05_LandingGear_Data_Generation",
    "02_06_Avionics_Data_Generation",
    "02_07_ElectricalSystems_Data_Generation",
    "02_08_FuelSystems_Data_Generation",
    "02_09_HydraulicSystems_Data_Generation",
    "02_10_EnvironmentalSystems_Data_Generation"
]

w = WorkspaceClient()
tasks = []

# ✅ Add synthetic data notebook tasks (no dependencies)
for notebook in synthetic_notebooks:
    notebook_path_full = f"{synthetic_data_path}/{notebook}"
    print(f"📍 Adding notebook task: {notebook_path_full}")
    tasks.append(Task(
        task_key=notebook,
        notebook_task=NotebookTask(notebook_path=notebook_path_full),
        existing_cluster_id=EXISTING_CLUSTER_ID if env == "dev" else None
    ))

# ✅ Add DLT pipeline task (depends on all synthetic tasks)
dlt_task_key = "Run_DLT_Pipeline"
tasks.append(Task(
    task_key=dlt_task_key,
    pipeline_task=PipelineTask(pipeline_id=DLT_PIPELINE_ID),
    depends_on=[TaskDependency(task_key=nb) for nb in synthetic_notebooks]
))
print(f"📍 Adding DLT pipeline task: Pipeline ID {DLT_PIPELINE_ID}")

# ✅ Add Feature Store registration notebook (depends on DLT)
feature_reg_task_key = "03B_Feature_Store_Registration"
feature_reg_path = f"{feature_registration_path}/03B_Feature_Store_Registration"
tasks.append(Task(
    task_key=feature_reg_task_key,
    notebook_task=NotebookTask(notebook_path=feature_reg_path),
    existing_cluster_id=EXISTING_CLUSTER_ID if env == "dev" else None,
    depends_on=[TaskDependency(task_key=dlt_task_key)]
))
print(f"📍 Adding Feature Store registration task: {feature_reg_path}")

# ✅ Add final summary notebook task (depends on feature registration)
final_task_key = "101_Final_Summary_Task"
final_task_path = f"{workflows_path}/{final_task_key}"
tasks.append(Task(
    task_key=final_task_key,
    notebook_task=NotebookTask(notebook_path=final_task_path),
    existing_cluster_id=EXISTING_CLUSTER_ID if env == "dev" else None,
    depends_on=[TaskDependency(task_key=feature_reg_task_key)]
))
print(f"📍 Adding final summary notebook task: {final_task_path}")

# ✅ Create or update the job
job = w.jobs.create(
    name=WORKFLOW_NAME,
    tasks=tasks
)

print(f"✅ Workflow '{WORKFLOW_NAME}' created/updated with Job ID: {job.job_id}")

In [0]:
job_id = job.job_id
print(f"✅ Workflow created/updated with Job ID: {job_id}")

In [0]:
import os

print("I am debugging  ============")

# Dynamically get the current notebook’s directory
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
folder_path = os.path.dirname(notebook_path)
workspace_base_path = f"/Workspace{folder_path}"

# Assume config folder is at the project root
project_root = "/".join(workspace_base_path.split("/")[:-1])  # remove last folder
config_file_path = f"{project_root}/config/aerodemo_config.py"

print(f"📁 Resolved config file path: {config_file_path}")

# LOCAL FILESYSTEM PATH (for Databricks Files API)
local_file_path = "/Workspace/Users/anand.rao@databricks.com/databricks-aerodemo/config/aerodemo_config.py"

if not os.path.isfile(local_file_path):
    print(f"❌ File does not exist at: {local_file_path}")
else:
    with open(local_file_path, "r") as file:
        lines = file.readlines()

    if not lines:
        print("⚠️ File is empty.")
    else:
        print(f"✅ File has {len(lines)} lines. Showing first 10 lines:")
        for idx, line in enumerate(lines[:10]):
            print(f"{idx + 1:03}: {repr(line.strip())}")

In [0]:
# Databricks FS path to repo
workspace_repo_path = "file:/Workspace/Repos/anand.rao@databricks.com/databricks-aerodemo/config/aerodemo_config.py"
local_temp_path = "/tmp/aerodemo_config.py"

try:
    dbutils.fs.cp(workspace_repo_path, f"file:{local_temp_path}", True)
    print(f"✅ Copied to local path: {local_temp_path}")

    with open(local_temp_path, "r") as f:
        content = f.read()
        print("✅ File content preview:")
        print(content[:500])  # just preview first 500 chars

except Exception as e:
    print(f"❌ Failed to copy or read file: {e}")

In [0]:
# import requests
# import json
# import re
# from config.aerodemo_config import get_config
# importlib.reload(aerodemo_config)  # Force reload of the module
# # Retrieve configuration for the current environment
# env = dbutils.widgets.get("ENV")
# config = get_config(env)

# # ---------- CONFIG ----------
# DATABRICKS_INSTANCE = config['databricks_instance']
# TOKEN = config['pat_token']
# JOB_ID = config['e2e_workflow_job_id']
# # ----------------------------
# print(f"✅ Using Databricks instance: {DATABRICKS_INSTANCE}")
# print(f"✅ Using PAT token: {TOKEN}")
# print(f"✅ Using job ID: {JOB_ID}")

In [0]:


# headers = {
#     "Authorization": f"Bearer {TOKEN}",
#     "Content-Type": "application/json"
# }

# # ✅ Step 1: Get the current job definition
# get_url = f"{DATABRICKS_INSTANCE}/api/2.1/jobs/get?job_id={JOB_ID}"
# response = requests.get(get_url, headers=headers)

# if response.status_code != 200:
#     print(f"❌ Failed to fetch job: {response.text}")
#     exit(1)

# job_data = response.json()
# print(f"✅ Fetched job '{job_data['settings']['name']}'")

# # ✅ Step 2: Strip '.ipynb' suffixes in notebook tasks
# for task in job_data['settings']['tasks']:
#     if 'notebook_task' in task and 'notebook_path' in task['notebook_task']:
#         original_path = task['notebook_task']['notebook_path']
#         cleaned_path = re.sub(r"\.ipynb$", "", original_path)
#         if original_path != cleaned_path:
#             print(f"🔧 Fixing: {original_path} → {cleaned_path}")
#             task['notebook_task']['notebook_path'] = cleaned_path

# # ✅ Step 3: Re-submit (reset) the job definition
# reset_url = f"{DATABRICKS_INSTANCE}/api/2.1/jobs/reset"
# payload = {
#     "job_id": JOB_ID,
#     "new_settings": job_data['settings']
# }

# reset_response = requests.post(reset_url, headers=headers, data=json.dumps(payload))

# if reset_response.status_code != 200:
#     print(f"❌ Failed to reset job: {reset_response.text}")
# else:
#     print(f"✅ Job '{job_data['settings']['name']}' successfully patched!")

In [0]:
# import requests
# import json
# from config.aerodemo_config import get_config

# # Retrieve configuration for the current environment
# env = dbutils.widgets.get("ENV")
# config = get_config(env)

# # ---------- CONFIG ----------
# DATABRICKS_INSTANCE = config['databricks_instance']
# TOKEN = config['pat_token']
# JOB_ID = config['e2e_workflow_job_id']
# DLT_PIPELINE_ID = config['workflow_configs']['dlt_pipeline_id']
# # ----------------------------

# headers = {
#     "Authorization": f"Bearer {TOKEN}",
#     "Content-Type": "application/json"
# }

# # ✅ Step 1: Get current job definition
# get_url = f"{DATABRICKS_INSTANCE}/api/2.1/jobs/get?job_id={JOB_ID}"
# response = requests.get(get_url, headers=headers)
# if response.status_code != 200:
#     print(f"❌ Failed to fetch job: {response.text}")
#     raise SystemExit

# job_data = response.json()
# print(f"✅ Fetched job '{job_data['settings']['name']}'")

# # ✅ Step 2: Check if DLT task already exists
# task_keys = [t['task_key'] for t in job_data['settings']['tasks']]
# if "Run_DLT_Pipeline" in task_keys:
#     print("⚠️ DLT task already exists in the workflow. Skipping add.")
# else:
#     # ✅ Step 3: Add DLT task at the end
#     job_data['settings']['tasks'].append({
#         "task_key": "Run_DLT_Pipeline",
#         "depends_on": [{"task_key": "02_10_EnvironmentalSystems_Data_Generation"}],
#         "pipeline_task": {
#             "pipeline_id": DLT_PIPELINE_ID
#         }
#     })
#     print("✅ DLT task added to workflow payload.")

#     # ✅ Step 4: Patch the updated workflow
#     reset_url = f"{DATABRICKS_INSTANCE}/api/2.1/jobs/reset"
#     payload = {
#         "job_id": JOB_ID,
#         "new_settings": job_data['settings']
#     }

#     reset_response = requests.post(reset_url, headers=headers, data=json.dumps(payload))
#     if reset_response.status_code != 200:
#         print(f"❌ Failed to patch job: {reset_response.text}")
#     else:
#         print(f"✅ Job '{job_data['settings']['name']}' successfully updated with DLT task!")

In [0]:
# import requests
# from config.aerodemo_config import get_config

# # Retrieve configuration for the current environment
# env = dbutils.widgets.get("ENV")
# config = get_config(env)

# # ---------- CONFIG ----------
# DATABRICKS_INSTANCE = config['databricks_instance']
# TOKEN = config['pat_token']
# JOB_ID = config['e2e_workflow_job_id']
# # ----------------------------

# headers = {
#     "Authorization": f"Bearer {TOKEN}",
#     "Content-Type": "application/json"
# }

# # ✅ Fetch the updated job definition
# get_url = f"{DATABRICKS_INSTANCE}/api/2.1/jobs/get?job_id={JOB_ID}"
# response = requests.get(get_url, headers=headers)
# if response.status_code != 200:
#     print(f"❌ Failed to fetch job: {response.text}")
#     raise SystemExit

# job_data = response.json()
# print(f"✅ Job '{job_data['settings']['name']}' has the following tasks:")
# for task in job_data['settings']['tasks']:
#     if 'notebook_task' in task:
#         print(f" - Notebook task: {task['task_key']} → {task['notebook_task']['notebook_path']}")
#     if 'pipeline_task' in task:
#         print(f" - DLT pipeline task: {task['task_key']} → Pipeline ID: {task['pipeline_task']['pipeline_id']}")